In [45]:
import fitz  # PyMuPDF
import re
import os

# --- CONFIGURACIÓN ---
DIRECTORIO_PDFS = "."
DIRECTORIO_IMAGENES = "imagenes_extraidas_final"
MATERIA_A_BUSCAR = "MAT1620"  # Puedes cambiar esto por ej: "FIS1514"
NOMBRE_ARCHIVO_FINAL = f"Compilado_{MATERIA_A_BUSCAR}_FINAL.pdf"

In [46]:
def mapear_y_capturar_preguntas(directorio, dir_imagenes):
    os.makedirs(dir_imagenes, exist_ok=True)
    archivos_pdf = [f for f in os.listdir(directorio) if f.startswith('Guia') and f.endswith('.pdf')]
    mapa_preguntas = []
    preguntas_finales = []

    print("--- PASO 1: Mapeando todas las preguntas... ---")
    docs_abiertos = {nombre: fitz.open(os.path.join(directorio, nombre)) for nombre in archivos_pdf}

    for nombre_archivo, doc in docs_abiertos.items():
        for num_pagina, pagina in enumerate(doc):
            instancias = pagina.search_for("Pregunta N°")
            for inst in instancias:
                mapa_preguntas.append({"fuente": nombre_archivo, "pagina": num_pagina, "bbox": inst})

    mapa_preguntas.sort(key=lambda p: (p["fuente"], p["pagina"], p["bbox"].y0))
    print(f"Mapeo completado. Se encontraron {len(mapa_preguntas)} inicios de preguntas.")
    
    print("\n--- PASO 2: Capturando cada pregunta como imagen... ---")
    for i, p_actual in enumerate(mapa_preguntas):
        doc = docs_abiertos[p_actual["fuente"]]
        pagina = doc[p_actual["pagina"]]
        
        inicio_rect = p_actual["bbox"]
        area_captura = fitz.Rect(inicio_rect.x0 - 20, inicio_rect.y0 - 10, pagina.rect.width - 20, pagina.rect.height)

        if i + 1 < len(mapa_preguntas) and mapa_preguntas[i+1]["fuente"] == p_actual["fuente"]:
            p_siguiente = mapa_preguntas[i+1]
            if p_siguiente["pagina"] == p_actual["pagina"]:
                area_captura.y1 = p_siguiente["bbox"].y0 - 10
        
        # Extraer el texto solo para obtener el número y la materia
        texto_header = pagina.get_text("text", clip=fitz.Rect(inicio_rect.x0, inicio_rect.y0, pagina.rect.width, inicio_rect.y0 + 50))
        num_match = re.search(r"Pregunta N°\s?(\d+)", texto_header)
        numero_pregunta = num_match.group(1) if num_match else f"idx{i}"
        materia_match = re.search(r"([A-Z]{3,7}\d{3,4})", texto_header)
        materia_sigla = materia_match.group(1) if materia_match else "Sin Materia"

        # Capturar el área como una imagen (Pixmap)
        pix = pagina.get_pixmap(clip=area_captura, dpi=300) # dpi alto para buena calidad
        nombre_imagen = f"{os.path.splitext(p_actual['fuente'])[0]}_P{numero_pregunta}.png"
        ruta_guardado = os.path.join(dir_imagenes, nombre_imagen)
        pix.save(ruta_guardado)

        preguntas_finales.append({
            "fuente": p_actual["fuente"],
            "numero": numero_pregunta,
            "materia": materia_sigla,
            "ruta_imagen": ruta_guardado
        })
        
    for doc in docs_abiertos.values(): doc.close()
    return preguntas_finales

def crear_pdf_con_imagenes(preguntas_db, codigo_materia_filtro, nombre_archivo_salida):
    preguntas_filtradas = [p for p in preguntas_db if p['materia'] == codigo_materia_filtro]
    
    if not preguntas_filtradas:
        print(f"\n❌ No se encontraron preguntas para la materia '{codigo_materia_filtro}'.")
        return

    print(f"\n--- PASO 3: Creando PDF para '{codigo_materia_filtro}' con {len(preguntas_filtradas)} preguntas... ---")
    doc = fitz.open()

    for pregunta in preguntas_filtradas:
        ruta_imagen = pregunta["ruta_imagen"]
        if os.path.exists(ruta_imagen):
            img = fitz.open(ruta_imagen)
            rect = img[0].rect
            page = doc.new_page(width=rect.width, height=rect.height)
            page.insert_image(rect, filename=ruta_imagen)
            img.close()
    
    try:
        doc.save(nombre_archivo_salida, garbage=4, deflate=True, clean=True)
        print(f"✅ ¡Éxito! Se ha creado el archivo: '{nombre_archivo_salida}'")
    except Exception as e:
        print(f"❌ Error al guardar PDF: {e}")
    finally:
        doc.close()

In [47]:
# Llamamos a la nueva función que captura imágenes en lugar de texto
lista_de_preguntas = mapear_y_capturar_preguntas(DIRECTORIO_PDFS, DIRECTORIO_IMAGENES)

if lista_de_preguntas:
    print(f"\n✅ PASOS 1 y 2 COMPLETADOS: Se procesaron y capturaron {len(lista_de_preguntas)} preguntas.")
else:
    print("\n❌ FALLÓ LA EXTRACCIÓN: No se encontraron preguntas.")

--- PASO 1: Mapeando todas las preguntas... ---
Mapeo completado. Se encontraron 642 inicios de preguntas.

--- PASO 2: Capturando cada pregunta como imagen... ---

✅ PASOS 1 y 2 COMPLETADOS: Se procesaron y capturaron 642 preguntas.


In [48]:
for k in lista_de_preguntas:
    print(k)

{'fuente': 'Guia de Ejercicios ECF 1_2016.pdf', 'numero': '1', 'materia': 'MAT1610', 'ruta_imagen': 'imagenes_extraidas_final\\Guia de Ejercicios ECF 1_2016_P1.png'}
{'fuente': 'Guia de Ejercicios ECF 1_2016.pdf', 'numero': '2', 'materia': 'MAT1620', 'ruta_imagen': 'imagenes_extraidas_final\\Guia de Ejercicios ECF 1_2016_P2.png'}
{'fuente': 'Guia de Ejercicios ECF 1_2016.pdf', 'numero': '3', 'materia': 'MAT1630', 'ruta_imagen': 'imagenes_extraidas_final\\Guia de Ejercicios ECF 1_2016_P3.png'}
{'fuente': 'Guia de Ejercicios ECF 1_2016.pdf', 'numero': '4', 'materia': 'MAT1640', 'ruta_imagen': 'imagenes_extraidas_final\\Guia de Ejercicios ECF 1_2016_P4.png'}
{'fuente': 'Guia de Ejercicios ECF 1_2016.pdf', 'numero': '5', 'materia': 'MAT1203', 'ruta_imagen': 'imagenes_extraidas_final\\Guia de Ejercicios ECF 1_2016_P5.png'}
{'fuente': 'Guia de Ejercicios ECF 1_2016.pdf', 'numero': '6', 'materia': 'MAT1203', 'ruta_imagen': 'imagenes_extraidas_final\\Guia de Ejercicios ECF 1_2016_P6.png'}
{'fu

In [49]:
# Generamos el PDF final a partir de las imágenes capturadas
if 'lista_de_preguntas' in locals() and lista_de_preguntas:
    crear_pdf_con_imagenes(lista_de_preguntas, MATERIA_A_BUSCAR, NOMBRE_ARCHIVO_FINAL)
else:
    print("No se puede generar el PDF porque la lista de preguntas está vacía.")


--- PASO 3: Creando PDF para 'MAT1620' con 10 preguntas... ---
✅ ¡Éxito! Se ha creado el archivo: 'Compilado_MAT1620_FINAL.pdf'
